import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480
cls_num=42
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=np.array(NonInvasiveFetalECGThorax1(i))

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (1800, 751)
first_test_shape= (1965, 751)
classes_quantity= 42
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42}
Count_labels= [36 52 45 40 48 44 36 50 46 56 36 39 46 42 40 39 52 45 40 49 43 40 35 41
 41 37 47 40 47 40 43 41 37 52 45 40 37 41 43 45 39 45]
max(train_feature_Altitude)= 42.0
min(train_feature_Altitude)= -5.7322181
first_train_sample=
 [ 3.3000000e+01  3.0577079e+00  2.9885631e+00  2.8214720e+00
  2.6392369e+00  2.4456015e+00  2.1598024e+00  1.7604417e+00
  1.2885524e+00  7.7969192e-01  2.5702086e-01 -2.4136675e-01
 -6.9723052e-01 -1.1682231e+00 -1.7182241e+00 -2.3618027e+00
 -3.0698550e+00 -3.7349237e+00 -4.2844303e+00 -4.7071089e+00
 -5.0354394e+00 -5.2173834e+00 -5.2658047e+00 -5.2337997e+00
 -5.1379266e+00 -5.0083663e+00 -4.8910492e+00 -4.7745101e+00
 -4.5631708e+00 -4.2865916e+00 -4.0250031e+00 -3.7163384e+00
 -3.3250977e+00 -2.957

C:\Users\user\AppData\Local\Temp/ipykernel_4776/3524553684.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=np.array(NonInvasiveFetalECGThorax1(i))


1 36	2 52	3 45	4 40	5 48	6 44	7 36	8 50	9 46	10 56	11 36	12 39	13 46	14 42	15 40	16 39	17 52	18 45	19 40	20 49	21 43	22 40	23 35	24 41	25 41	26 37	27 47	28 40	29 47	30 40	31 43	32 41	33 37	34 52	35 45	36 40	37 37	38 41	39 43	40 45	41 39	42 45	

In [3]:
np.shape(ecg)

(1800, 481)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [4]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 36	2 52	3 45	4 40	5 48	6 44	7 36	8 50	9 46	10 56	11 36	12 39	13 46	14 42	15 40	16 39	17 52	18 45	19 40	20 49	21 43	22 40	23 35	24 41	25 41	26 37	27 47	28 40	29 47	30 40	31 43	32 41	33 37	34 52	35 45	36 40	37 37	38 41	39 43	40 45	41 39	42 45	
 max = 56


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [5]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (36, 481)
Up to class  2 train shape =  (88, 481)
Up to class  3 train shape =  (133, 481)
Up to class  4 train shape =  (173, 481)
Up to class  5 train shape =  (221, 481)
Up to class  6 train shape =  (265, 481)
Up to class  7 train shape =  (301, 481)
Up to class  8 train shape =  (351, 481)
Up to class  9 train shape =  (397, 481)
Up to class  10 train shape =  (453, 481)
Up to class  11 train shape =  (489, 481)
Up to class  12 train shape =  (528, 481)
Up to class  13 train shape =  (574, 481)
Up to class  14 train shape =  (616, 481)
Up to class  15 train shape =  (656, 481)
Up to class  16 train shape =  (695, 481)
Up to class  17 train shape =  (747, 481)
Up to class  18 train shape =  (792, 481)
Up to class  19 train shape =  (832, 481)
Up to class  20 train shape =  (881, 481)
Up to class  21 train shape =  (924, 481)
Up to class  22 train shape =  (964, 481)
Up to class  23 train shape =  (999, 481)
Up to class  24 train shape =  (1040, 481)
Up

In [6]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 36 52 45 40 48 44 36 50 46 56 36 39 46 42 40 39 52 45 40 49 43 40 35
 41 41 37 47 40 47 40 43 41 37 52 45 40 37 41 43 45 39 45]


In [7]:
xtrain[:,-1]

array([ 1.,  1.,  1., ..., 42., 42., 42.])

In [8]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [9]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [10]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [11]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  3.464960350576406
min magnitude class 1  =  -4.0359667553513825
after normalizing >>
max magnitude class 1  =  0.7551004095379115
min magnitude class 1  =  -0.6763971402363305

 cls 2  >> 
max magnitude class 2  =  4.425310812468809
min magnitude class 2  =  -3.0005668376799934
after normalizing >>
max magnitude class 2  =  0.9383763317951899
min magnitude class 2  =  -0.47879857351892796

 cls 3  >> 
max magnitude class 3  =  3.498304696858196
min magnitude class 3  =  -4.053376157408748
after normalizing >>
max magnitude class 3  =  0.7614639362496272
min magnitude class 3  =  -0.6797195983853317

 cls 4  >> 
max magnitude class 4  =  3.6561362518347744
min magnitude class 4  =  -3.8223670751843044
after normalizing >>
max magnitude class 4  =  0.7915849440977742
min magnitude class 4  =  -0.6356331900640786

 cls 5  >> 
max magnitude class 5  =  4.072548295052668
min magnitude class 5  =  -3.356934300988409
after normalizing >>
max magnitude cl

min magnitude class 42  =  -0.39992030217882557
each class and its windows =
1 48	2 39	3 42	4 45	5 48	6 50	7 57	8 41	9 42	10 36	11 39	12 51	13 51	14 47	15 58	16 56	17 33	18 46	19 48	20 41	21 46	22 46	23 52	24 46	25 49	26 41	27 42	28 45	29 49	30 46	31 51	32 49	33 48	34 42	35 49	36 51	37 50	38 50	39 47	40 46	41 54	42 48	
 max instance in classes of test windows = 58


In [12]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.0014654308353825
max tst = 1.1580859210715708


In [13]:
len(wndws_test)

1965

In [14]:
np.shape(wndws_test)

(1965, 481)

In [15]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [16]:
fullprint(ytest)

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  6,  6,  6,  6

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [17]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [18]:
np.shape(wndws1)

(36, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [19]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [20]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (1800, 480)
y_train =>  (1800,)
X_test  =>  (1965, 480)
y_test  =>  (1965,)
X_valid  =>  (1800, 480)
y_valid  =>  (1800,)


In [21]:
y_train=np.array(ytrain)
np.min(y_train)

1

In [22]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [23]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
23/23 [==============================] - 5s 57ms/step - loss: 4.1059 - accuracy: 0.0372 - val_loss: 3.7546 - val_accuracy: 0.0250
Epoch 2/1000
23/23 [==============================] - 1s 23ms/step - loss: 3.0439 - accuracy: 0.1328 - val_loss: 3.7424 - val_accuracy: 0.0339
Epoch 3/1000
23/23 [==============================] - 1s 24ms/step - loss: 2.4858 - accuracy: 0.2467 - val_loss: 3.7132 - val_accuracy: 0.0783
Epoch 4/1000
23/23 [==============================] - 1s 24ms/step - loss: 1.9771 - accuracy: 0.3578 - val_loss: 3.7087 - val_accuracy: 0.0672
Epoch 5/1000
23/23 [==============================] - 1s 24ms/step - loss: 1.6524 - accuracy: 0.4489 - val_loss: 3.6884 - val_accuracy: 0.0611
Epoch 6/1000
23/23 [==============================] - 1s 23ms/step - loss: 1.3501 - accuracy: 0.5461 - val_loss: 3.7149 - val_accuracy: 0.0289
Epoch 7/1000
23/23 [==============================] - 1s 24ms/step - loss: 1.1363 - accuracy: 0.6200 - val_loss: 3.7074 - val_accuracy: 0.0472

Epoch 58/1000
23/23 [==============================] - 1s 23ms/step - loss: 0.0874 - accuracy: 0.9739 - val_loss: 0.0854 - val_accuracy: 0.9706
Epoch 59/1000
23/23 [==============================] - 1s 24ms/step - loss: 0.1277 - accuracy: 0.9733 - val_loss: 0.1227 - val_accuracy: 0.9661
Epoch 60/1000
23/23 [==============================] - 1s 23ms/step - loss: 0.0732 - accuracy: 0.9778 - val_loss: 0.3691 - val_accuracy: 0.9294
Epoch 61/1000
23/23 [==============================] - 1s 24ms/step - loss: 0.1003 - accuracy: 0.9767 - val_loss: 0.0787 - val_accuracy: 0.9728
Epoch 62/1000
23/23 [==============================] - 1s 23ms/step - loss: 0.1039 - accuracy: 0.9794 - val_loss: 0.0367 - val_accuracy: 0.9872
Epoch 63/1000
23/23 [==============================] - 1s 24ms/step - loss: 0.0855 - accuracy: 0.9789 - val_loss: 0.5262 - val_accuracy: 0.8500
Epoch 64/1000
23/23 [==============================] - 1s 23ms/step - loss: 0.0635 - accuracy: 0.9806 - val_loss: 0.0357 - val_accuracy:

In [24]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [25]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
print(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

62/62 [==============================] - 0s 5ms/step
[[38  0  1 ...  0  0  0]
 [ 0 38  0 ...  0  0  0]
 [ 2  0 13 ...  0  0  0]
 ...
 [ 0  0  0 ... 38  0  0]
 [ 0  0  0 ...  0 42  0]
 [ 0  0  0 ...  0  0 48]]
              precision    recall  f1-score   support

           1       0.93      0.79      0.85        48
           2       1.00      0.97      0.99        39
           3       0.48      0.31      0.38        42
           4       0.85      0.49      0.62        45
           5       1.00      0.85      0.92        48
           6       0.75      0.80      0.78        50
           7       0.96      0.96      0.96        57
           8       0.84      1.00      0.91        41
           9       0.98      0.95      0.96        42
          10       0.97      1.00      0.99        36
          11       0.83      1.00      0.91        39
          12       1.00      1.00      1.00        51
          13       0.98      0.88      0.93        51
          14       0.80      0.87 